# Introduction - Problem Statement & Approach

This Jupyter Notebook processes [Emergency Veterinarian Notification Services (EVNS) data](https://docs.google.com/spreadsheets/d/1AQ0L7I1yKA0zCpOyCj6lsf0kiLRLlEDKetUaZGnriqg/edit?usp=sharing). 

It will:
1. Load sample data from the provided Excel file into a PostgreSQL database on AWS.
2. Explore sample data.
3. Run SQL query to generate the emergency alert data format.
4. Use Python to filter results based on the `Vet Targets` file and output the final processed data file.

#### Assumptions: 

a. The `VETS` table is our source-of-truth (being internal, verified data), and missing vet data should be backfilled from `EXTERNAL_VET_DATA` which is our second-best source-of-truth.

b. We are only able to contact users from the `VETS` table.

# Key Takeaways

* The final output file has 47 records, 36 unique vet IDs, and 4 records missing city and state.
* "None" and "Other" specialties are included since the "Vet Targets" list likely came from a client who knows exactly who they want to target, regardless of the specialty we have on file. It would be good to confirm with stakeholders how they would like to handle this case going forward.
* It appears that the `vet_id` field from the `VET` table is the most accurate and should be matched to the target file.
* Some records in `VET_INFO` and `VET` tables have `NaN` values for critical fields such as `vet_id`, `first_name`, `last_name`, and `specialty`, which may impact analyses requiring a complete dataset.
* The various zip code values are sometimes missing or unpadded. (They don't seem to match city/state, either, but I assume it's the nature of the synthetic data for this assignment). 

## Recommended Next Steps


* Explore backfill methods in the `VET` table for critical fields (i.e. `vet_id`, `zip_code`, `specialty`).
* Deduplicate records where necessary, especially in `VET` and `VET_INFO` tables. (Note: If a user has more than one `vet_id`, then we can add a `primary` field to associate with their "best" ID. This will help ensure consistency with targeting and any ROI studies focusing on Rx lift.)
* Productionalize pipeline by automating data ingestion, validation, and transformation using scheduled ETL processes, enforcing schema consistency, and implementing monitoring to detect missing or inconsistent data in real time.


In [44]:
%pip install psycopg2
%pip install pandas
%pip install openpyxl
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [50]:
import pandas as pd
import psycopg2
from sqlalchemy import Integer, Boolean, String, UUID, BigInteger, DateTime, create_engine
from IPython.display import display 

# 1. Load sample data from the provided Excel file into a PostgreSQL database on AWS.

In [51]:
# Load Excel file with multiple sheets
file_path = "~/Downloads/MDCalc_Data_Exercise.xlsx"
sheets = pd.ExcelFile(file_path)

# Define a converter function for "do_not_sell" since Pandas is misinterpreting it
def bool_converter(value):
    if isinstance(value, bool):  # If already a boolean, keep it
        return value
    if str(value).strip().lower() in ["false", "0", "no"]:  # Standardize false values
        return False
    if str(value).strip().lower() in ["true", "1", "yes"]:  # Standardize true values
        return True
    return None  # Handle empty or unexpected values

# Define correct data types for each sheet (EXCLUDING "do_not_sell")
dtype_mapping = {
    "vet_id": "Int64",  # Ensures integer storage, allows NaNs
    "uuid": "string",  # Keep UUIDs as strings
    "vet_id_source": "string",
    "first_name": "string",
    "last_name": "string",
    "zip": "string",
    "zip_code": "string",
    "specialty": "string",
    "address": "string",
    "city": "string",
    "state": "string",
    "publisher_deal_id": "string", #Although this is BigInt in data dictionary, values should be text as they contain a dash
    "event": "string",
}

# Load specific sheets into DataFrames with enforced data types
vet_targets_data = sheets.parse("Vet Targets", dtype=dtype_mapping)
vet_info_data = sheets.parse("vet_info", dtype=dtype_mapping, converters={"do_not_sell": bool_converter})
vet_data = sheets.parse("vet", dtype=dtype_mapping)
vet_emergency_alerts_log_data = sheets.parse(
    "vet_emergecy_alerts_log", dtype=dtype_mapping, parse_dates=["timestamp"]
)  
vet_emergency_subscription_data = sheets.parse("vet_emergency_subscription", dtype=dtype_mapping)
external_vet_data_data = sheets.parse("external_vet_data", dtype=dtype_mapping)


# Preview datatypes
print("\n--- Vet Targets Data Types ---\n", vet_targets_data.dtypes)
print("\n--- Vet Info Data Types ---\n", vet_info_data.dtypes)
print("\n--- Vet Data Types ---\n", vet_data.dtypes)
print("\n--- Vet Emergency Alerts Log Data Types ---\n", vet_emergency_alerts_log_data.dtypes)
print("\n--- Vet Emergency Subscription Data Types ---\n", vet_emergency_subscription_data.dtypes)
print("\n--- External Vet Data Types ---\n", external_vet_data_data.dtypes)

# Preview data
print("\n--- Vet Targets Data Preview ---\n", vet_targets_data.head())
print("\n--- Vet Info Data Preview ---\n", vet_info_data.head())
print("\n--- Vet Data Preview ---\n", vet_data.head())
print("\n--- Vet Emergency Alerts Log Data Preview ---\n", vet_emergency_alerts_log_data.head())
print("\n--- Vet Emergency Subscription Data Preview ---\n", vet_emergency_subscription_data.head())
print("\n--- External Vet Data Preview ---\n", external_vet_data_data.head())




--- Vet Targets Data Types ---
 vet_id    Int64
dtype: object

--- Vet Info Data Types ---
 uuid             string[python]
vet_id                    Int64
vet_id_source    string[python]
do_not_sell                bool
dtype: object

--- Vet Data Types ---
 uuid          string[python]
vet_id                 Int64
first_name    string[python]
last_name     string[python]
zip_code      string[python]
specialty     string[python]
dtype: object

--- Vet Emergency Alerts Log Data Types ---
 uuid                                    string[python]
vet_emergency_subscription_uuid                 object
vet_uuid                                        object
event                                   string[python]
timestamp                          datetime64[ns, UTC]
dtype: object

--- Vet Emergency Subscription Data Types ---
 uuid                 string[python]
subscription_name            object
publisher                    object
publisher_deal_id    string[python]
dtype: object

--- Extern

In [52]:
# Connect to database

# Credentials
USER = "postgres"
PASSWORD = "$mdcalc!" # Hardcoding passwords is a security risk, and I would use environment variables in a real scenario. 
AWS_ENDPOINT = "mydbinstance.cnuuqekuqqgd.us-west-2.rds.amazonaws.com"
PORT = "5432"
DATABASE = "data_exercises"

# Use SSL (since AWS enforces it)
DATABASE_URL = f"postgresql://{USER}:{PASSWORD}@{AWS_ENDPOINT}:{PORT}/{DATABASE}?sslmode=require"

# Create the database engine
engine = create_engine(DATABASE_URL)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connected to AWS PostgreSQL successfully!")
except Exception as e:
    print("Error connecting to PostgreSQL:", e)


Connected to AWS PostgreSQL successfully!


In [53]:
# Load data into PostgreSQL with enforced data types
# NOTE: We are working with small amounts of data, so there is no need to chunk and insert in batches.

vet_targets_data.to_sql(
    "vet_targets", con=engine, if_exists="replace", index=False
)

vet_info_data.to_sql(
    "vet_info",
    con=engine,
    if_exists="replace",
    index=False,
    dtype={
        "uuid": UUID(),
        "vet_id": Integer(),
        "do_not_sell": Boolean(),
        "vet_id_source": String(25),
    }
)

vet_data.to_sql(
    "vet",
    con=engine,
    if_exists="replace",
    index=False,
    dtype={
        "uuid": UUID(),
        "vet_id": Integer(),
        "first_name": String(100),
        "last_name": String(100),
        "zip_code": String(100),
        "specialty": String(100),
    }
)

vet_emergency_alerts_log_data.to_sql(
    "vet_emergency_alerts_log",
    con=engine,
    if_exists="replace",
    index=False,
    dtype={
        "uuid": UUID(),
        "vet_uuid": UUID(),
        "vet_emergency_subscription_uuid": UUID(),
        "event": String(100),
        "event_timestamp": DateTime(timezone=True),
    }
)

vet_emergency_subscription_data.to_sql(
    "vet_emergency_subscription",
    con=engine,
    if_exists="replace",
    index=False,
    dtype={
        "uuid": UUID(),
        "subscription_name": String(100),
        "publisher": String(100),
        "publisher_deal_id": String(100),
    }
)

external_vet_data_data.to_sql(
    "external_vet_data",
    con=engine,
    if_exists="replace",
    index=False,
    dtype={
        "vet_id": Integer(),
        "first_name": String(100),
        "last_name": String(100),
        "address": String(100),
        "city": String(100),
        "state": String(100),
        "zip": String(100),
        "specialty": String(100),
    }
)

print("Data loaded successfully!")

Data loaded successfully!


# 2. EDA

In [54]:
# Check out new tables

# Print available tables
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
tables = pd.read_sql(query, con=engine)
print("\n--- List of Tables in the Database ---\n", tables)

for table_name in tables["table_name"]:  # Extract table names correctly
    print(f"\n--- Structure of `{table_name}` ---\n")
    
    # Describe the table structure (column names & types)
    describe_query = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}';"
    table_info = pd.read_sql(describe_query, con=engine)
    
    print(table_info)



--- List of Tables in the Database ---
                    table_name
0                 vet_targets
1                    vet_info
2                         vet
3    vet_emergency_alerts_log
4  vet_emergency_subscription
5           external_vet_data
6     vet_emergecy_alerts_log

--- Structure of `vet_targets` ---

  column_name data_type
0      vet_id    bigint

--- Structure of `vet_info` ---

     column_name          data_type
0           uuid               uuid
1         vet_id            integer
2    do_not_sell            boolean
3  vet_id_source  character varying

--- Structure of `vet` ---

  column_name          data_type
0        uuid               uuid
1      vet_id            integer
2  first_name  character varying
3   last_name  character varying
4    zip_code  character varying
5   specialty  character varying

--- Structure of `vet_emergency_alerts_log` ---

                       column_name                 data_type
0                             uuid               

In [55]:
#Vet Info
query = """
    SELECT 
        V.uuid AS vet_uuid,
        V.vet_id AS vet_vet_id,
        V.first_name,
        V.last_name,
        V.zip_code,
        V.specialty,
        VI.uuid AS vet_info_uuid,
        VI.vet_id AS vet_info_vet_id,
        VI.vet_id_source,
        VI.do_not_sell
    FROM VET V
    FULL OUTER JOIN VET_INFO VI 
    ON V.uuid = VI.uuid;
"""
int_vet_df = pd.read_sql(query, con=engine)
int_vet_df



,vet_uuid,vet_vet_id,first_name,last_name,zip_code,specialty,vet_info_uuid,vet_info_vet_id,vet_id_source,do_not_sell
0,None,NaN,None,None,None,None,cdaff8e5-b977-4b22-88e3-8431fb495732,1932125259,verified,False
1,f59b6df8-eeaf-fc03-2ded-d8658752dcff,NaN,Madison,Seda,7070,Surgery,f59b6df8-eeaf-fc03-2ded-d8658752dcff,1972068484,verified,False
2,f36568dd-4187-400c-b0b4-497db667c2b2,1.104353e+09,Saurav,Wash,None,Canine and Feline,f36568dd-4187-400c-b0b4-497db667c2b2,1104353424,verified,False
3,None,NaN,None,None,None,None,ac2b7bdc-7306-4203-ba54-ab0e74566403,1104820133,verified,False
4,None,NaN,None,None,None,None,34a9cbc1-4899-4d4f-8922-fbe4d9978b57,1053413443,verified,False
...,...,...,...,...,...,...,...,...,...,...
509,f471f141-2775-4a3d-9724-363e2cc6bdaf,1.861020e+09,Rasendu,Metzger,30068,Canine and Feline,f471f141-2775-4a3d-9724-363e2cc6bdaf,1861019879,verified,False
510,None,NaN,None,None,None,None,c6a8ce09-bace-44ca-b851-9e0681e5c471,1518260199,verified,False
511,None,NaN,None,None,None,None,1ff5679c-866a-409a-89e7-5d09287c9cec,1467422618,verified,False
512,9fcd99fe-f3be-453f-87bf-5c478e0f27be,NaN,Owen,Perez,20708,None,9fcd99fe-f3be-453f-87bf-5c478e0f27be,1649314311,verified,False


In [56]:
# Count records that exist only in VET 
only_in_vet = int_vet_df[int_vet_df["vet_info_uuid"].isna()].shape[0]

# Count records that exist only in VET_INFO 
only_in_vet_info = int_vet_df[int_vet_df["vet_uuid"].isna()].shape[0]

# Count records that exist in BOTH VET and VET_INFO
in_both = int_vet_df[int_vet_df["vet_uuid"].notna() & int_vet_df["vet_info_uuid"].notna()].shape[0]

# Print results
print(f"Records only in VET: {only_in_vet}")
print(f"Records only in VET_INFO: {only_in_vet_info}")
print(f"Records in BOTH tables: {in_both}")

# ✅ Raise an error if in_both + only_in_vet_info is not 514
if in_both + only_in_vet_info != 514:
    raise ValueError("Error: The sum of 'in_both' and 'only_in_vet_info' is NOT 514, which is unexpected!")


Records only in VET: 0
Records only in VET_INFO: 294
Records in BOTH tables: 220


In [57]:
# Inspect records that are only in VET_INFO. 
only_in_vet_info_df = int_vet_df[int_vet_df["vet_uuid"].isna()]

# Are there verified vets that we can contact?
only_in_vet_pivot = only_in_vet_info_df.pivot_table(
    index=["do_not_sell", "vet_id_source"], 
    values="vet_uuid", 
    aggfunc="count"
)
only_in_vet_pivot

,,vet_uuid
do_not_sell,vet_id_source,
False,verified,0


*All missing vet details are associated with contactable vets, so it is worth trying to backfill.* 

In [58]:
#Backfill missing data where available; prioritize data in `VETS` table. 

#Vet Info
backfill_query = """
    SELECT 
        VI.uuid, 
        VI.vet_id, 
        VI.vet_id_source,
        VI.do_not_sell, 
        COALESCE(V.first_name, EV.first_name) as first_name,
        COALESCE(V.last_name, EV.last_name) as last_name,
        EV.city,
        EV.state,
        COALESCE(V.zip_code, EV.zip) as zip,
        COALESCE(V.specialty, EV.specialty) as specialty
    FROM VET_INFO VI 
    LEFT JOIN VET V ON V.uuid = VI.uuid
    LEFT JOIN EXTERNAL_VET_DATA EV ON EV.vet_id = VI.vet_id;
"""
backfill_vet_df = pd.read_sql(backfill_query, con=engine)
backfill_vet_df


,uuid,vet_id,vet_id_source,do_not_sell,first_name,last_name,city,state,zip,specialty
0,cdaff8e5-b977-4b22-88e3-8431fb495732,1932125259,verified,False,None,None,None,None,None,None
1,f59b6df8-eeaf-fc03-2ded-d8658752dcff,1972068484,verified,False,Madison,Seda,None,None,7070,Surgery
2,f36568dd-4187-400c-b0b4-497db667c2b2,1104353424,verified,False,Saurav,Wash,None,None,None,Canine and Feline
3,ac2b7bdc-7306-4203-ba54-ab0e74566403,1104820133,verified,False,None,None,None,None,None,None
4,34a9cbc1-4899-4d4f-8922-fbe4d9978b57,1053413443,verified,False,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
509,f471f141-2775-4a3d-9724-363e2cc6bdaf,1861019879,verified,False,Rasendu,Metzger,None,None,30068,Canine and Feline
510,c6a8ce09-bace-44ca-b851-9e0681e5c471,1518260199,verified,False,None,None,None,None,None,None
511,1ff5679c-866a-409a-89e7-5d09287c9cec,1467422618,verified,False,None,None,None,None,None,None
512,9fcd99fe-f3be-453f-87bf-5c478e0f27be,1649314311,verified,False,Owen,Perez,None,None,20708,None


In [60]:
# Count the number of null values in each column
null_counts = backfill_vet_df.isnull().sum()

# Convert to DataFrame for better readability
null_counts_df = pd.DataFrame(null_counts, columns=['null_count'])
null_counts_df

,null_count
uuid,0
vet_id,0
vet_id_source,0
do_not_sell,0
first_name,294
last_name,294
city,514
state,514
zip,302
specialty,339


*The `EXTERNAL_VET_DATA` join is not pulling in any data. Let's see why this is.*

In [14]:
#Explore join issue 
root_cause_query = """
    SELECT 
        V.uuid AS vet_and_info_uuid,
        V.vet_id AS vet_vet_id,
        VI.vet_id AS info_vet_id,
        EV.vet_id AS external_vet_id
    FROM VET_INFO VI  
    LEFT JOIN VET V ON V.uuid = VI.uuid
    LEFT JOIN EXTERNAL_VET_DATA EV ON EV.vet_id = V.vet_id
    LEFT JOIN EXTERNAL_VET_DATA EV2 ON EV.vet_id = VI.vet_id
    --WHERE EV.vet_id = '12850960';
    WHERE V.vet_id != VI.vet_id OR EV.vet_id != VI.vet_id;
"""

root_cause_df = pd.read_sql(root_cause_query, con=engine)
root_cause_df

,vet_and_info_uuid,vet_vet_id,info_vet_id,external_vet_id
0,f36568dd-4187-400c-b0b4-497db667c2b2,1104353487,1104353424,1104353487
1,93828087-26af-480f-8ebb-88b6bd815983,10851572,1063851509,10851572
2,f6356146-4325-4329-bf61-fa3050c90b43,1316924158,1316924095,1316924158
3,e1e266a5-c3d4-493e-a8a3-d4247d456a33,1790924032,1790923969,1790924032
4,cfd048f9-80d6-4f65-a2b0-a803823fd318,1093074333,1093074270,1093074333
...,...,...,...,...
176,b97ce303-9c15-4694-b89d-62ec5bcfc7fe,1912534658,1912534595,1912534658
177,72858752-11ce-46e2-bc83-e92be53e1a9d,1013302278,1013302215,1013302278
178,d018ba44-501e-76a7-6d88-c144c2906404,1538392176,1538392113,1538392176
179,f471f141-2775-4a3d-9724-363e2cc6bdaf,1861019942,1861019879,1861019942


*As we can see above, the same vet UUID can be associated with different vet IDs. This does not appear to be a padding or rounding issue, but rather actual different values. If these were NPI numbers, I would find this very concerning. While it's true that a prescriber could have more than one NPI, it is not so common that I would so many records to point to different IDs. Let's explore some more to see what's going on here.*

In [65]:
# Check out duplicate and missing values 

def count_duplicates_and_missing(df, table_name, columns):
    """
    Counts duplicate and missing values for specific columns in a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to analyze.
        table_name (str): Name of the table (for display purposes).
        columns (list): List of column names to check for duplicates and missing values.

    Returns:
        pd.DataFrame: A DataFrame showing duplicate and missing counts per column.
    """
    results = []
    
    for col in columns:
        if col in df.columns:
            duplicate_count = df[col].duplicated(keep=False).sum()
            missing_count = df[col].isna().sum()
            results.append({"Column": col, "Duplicate Count": duplicate_count, "Missing Count": missing_count})

    # Convert to DataFrame for display
    results_df = pd.DataFrame(results)

    # Display results
    print(f"\n--- {table_name} ---\n")
    display(results_df)

# Run the function for each table and display results
count_duplicates_and_missing(vet_data, "vet", ["vet_id", "uuid"])
count_duplicates_and_missing(vet_info_data, "vet_info", ["vet_id", "uuid"])
count_duplicates_and_missing(external_vet_data_data, "external_vet_data", ["vet_id"])



--- vet ---



,Column,Duplicate Count,Missing Count
0,vet_id,39,39
1,uuid,0,0



--- vet_info ---



,Column,Duplicate Count,Missing Count
0,vet_id,10,0
1,uuid,0,0



--- external_vet_data ---



,Column,Duplicate Count,Missing Count
0,vet_id,2,0


In [67]:
# Does the external data match any vet_id values in the VET_INFO table? 
# Which join has the better match rate?

HELP_QUERY = """
    SELECT COUNT(DISTINCT VI.VET_ID) AS VI_VET_ID_MATCH_COUNT
    FROM VET_INFO VI 
    INNER JOIN VET_TARGETS VT ON VT.VET_ID = VI.VET_ID 
    UNION ALL 
    SELECT COUNT(DISTINCT V.VET_ID) AS V_VET_ID_MATCH_COUNT
    FROM VET V
    INNER JOIN VET_TARGETS VT ON VT.VET_ID = V.VET_ID
    """
HELP_QUERY_df = pd.read_sql(HELP_QUERY, con=engine)
HELP_QUERY_df

,vi_vet_id_match_count
0,0
1,105


*We see that some vet IDs are duplicated even though UUIDs are not, and we also see that no `vet_id` values from the `VET_INFO` table match the `VET_TARGETS` table. It is not clear what this `vet_id` value represents and what expectations are around matching. More work should be done to understand this ID issue. For this project, we will assume that the `vet_id` from the `VET` table is the most accurate and should be matched to the target file.*

In [68]:
# Incorporate new understanding into backfill query 

final_backfill_query = """
    SELECT 
        VI.uuid, 
        V.vet_id, -- no VI.vet_ids match the targeting file
        VI.vet_id_source,
        COALESCE(V.first_name, EV.first_name) as first_name,
        COALESCE(V.last_name, EV.last_name) as last_name,
        EV.city,
        EV.state,
        COALESCE(V.zip_code, EV.zip) as zip,
        COALESCE(V.specialty, EV.specialty) as specialty, 
        VI.do_not_sell
    FROM VET_INFO VI 
    LEFT JOIN VET V ON V.uuid = VI.uuid -- good join on foriegn keys
    LEFT JOIN EXTERNAL_VET_DATA EV ON EV.vet_id = V.vet_id -- assuming we can trust this join as most values align rather than VET_INFO
    ;
"""
final_backfill_vet_df = pd.read_sql(final_backfill_query, con=engine)
final_backfill_vet_df

,uuid,vet_id,vet_id_source,first_name,last_name,city,state,zip,specialty,do_not_sell
0,cdaff8e5-b977-4b22-88e3-8431fb495732,NaN,verified,None,None,None,None,None,None,False
1,f59b6df8-eeaf-fc03-2ded-d8658752dcff,NaN,verified,Madison,Seda,None,None,7070,Surgery,False
2,f36568dd-4187-400c-b0b4-497db667c2b2,1.104353e+09,verified,Saurav,Wash,None,None,None,Canine and Feline,False
3,ac2b7bdc-7306-4203-ba54-ab0e74566403,NaN,verified,None,None,None,None,None,None,False
4,34a9cbc1-4899-4d4f-8922-fbe4d9978b57,NaN,verified,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...
509,f471f141-2775-4a3d-9724-363e2cc6bdaf,1.861020e+09,verified,Rasendu,Metzger,Duluth,GA,30068,Canine and Feline,False
510,c6a8ce09-bace-44ca-b851-9e0681e5c471,NaN,verified,None,None,None,None,None,None,False
511,1ff5679c-866a-409a-89e7-5d09287c9cec,NaN,verified,None,None,None,None,None,None,False
512,9fcd99fe-f3be-453f-87bf-5c478e0f27be,NaN,verified,Owen,Perez,None,None,20708,None,False


In [69]:
# Tie in events and subscription data

event_and_sub_query = """
    SELECT 
        V.uuid, 
        V.vet_id, -- no VI.vet_ids match the targeting file
        VI.vet_id_source,
        COALESCE(V.first_name, EV.first_name) as first_name,
        COALESCE(V.last_name, EV.last_name) as last_name,
        EV.city,
        EV.state,
        COALESCE(V.zip_code, EV.zip) as zip,
        COALESCE(V.specialty, EV.specialty) as specialty, 
        VI.do_not_sell, 
        AL.uuid as log_uuid,
        AL.event,
        AL.timestamp AS event_timestamp,
        ES.subscription_name, 
        ES.publisher, 
        ES.publisher_deal_id

    FROM VET_INFO VI 
    INNER JOIN VET V ON V.uuid = VI.uuid -- good join on foriegn keys
    LEFT JOIN EXTERNAL_VET_DATA EV ON EV.vet_id = V.vet_id -- assuming we can trust this join as most values align rather than VET_INFO
    INNER JOIN VET_EMERGENCY_ALERTS_LOG AL ON AL.vet_uuid = V.uuid
    INNER JOIN VET_EMERGENCY_SUBSCRIPTION ES on ES.uuid = AL.vet_emergency_subscription_uuid; 
    """

event_and_sub_df = pd.read_sql(event_and_sub_query, con=engine)
event_and_sub_df


,uuid,vet_id,vet_id_source,first_name,last_name,city,state,zip,specialty,do_not_sell,log_uuid,event,event_timestamp,subscription_name,publisher,publisher_deal_id
0,26a94c8e-7f88-4da4-8631-a1b326706b7f,NaN,verified,Andrew,Ma,None,None,6850,Clinical pathology,False,0718c3cb-a900-4d6a-8d5b-f479ba445d41,notified,2025-02-16 19:06:58.729598+00:00,EM_VETS_12,EM Vets Chicago,14582726-111
1,26a94c8e-7f88-4da4-8631-a1b326706b7f,NaN,verified,Andrew,Ma,None,None,6850,Clinical pathology,False,c5a12163-9f3e-46be-abca-e3f6542fcc80,rejected,2025-02-16 19:07:00.251189+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
2,f6356146-4325-4329-bf61-fa3050c90b43,1.316924e+09,verified,Ebon,Ajibola,Columbus,OH,43004,Canine and Feline,False,da35ec1a-f097-4622-a692-0b10ed568830,notified,2025-02-15 16:44:06.359610+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
3,e1e266a5-c3d4-493e-a8a3-d4247d456a33,1.790924e+09,verified,Bruce,Calikyan,Auburn,CA,58104,Immunology,False,9e4692b9-337b-49ba-8342-c7376ad13629,notified,2025-02-15 23:11:45.401655+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
4,bdb25f86-273f-4a76-a709-41795401c1d7,NaN,verified,Heather,Chace,None,None,24060,None,False,a7c44bce-34cb-42b2-a6ee-57c5967e6512,notified,2025-02-15 22:15:17.259836+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,5453b1ec-3e0a-6748-593e-5d3a817540b5,NaN,verified,Robert,Smith,None,None,98292,None,False,c5eb8d08-0b07-48fb-9145-728167f9c3f9,notified,2025-02-15 16:20:57.882313+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
107,72858752-11ce-46e2-bc83-e92be53e1a9d,1.013302e+09,verified,Shawn,Suhrer,None,None,92354,Canine and Feline,False,46ca56ea-6f52-41eb-b21f-79a8da4a02bf,accepted,2025-02-15 21:33:27.630948+00:00,EM_VETS_17,EM Vets Washington,14582726-116
108,72858752-11ce-46e2-bc83-e92be53e1a9d,1.013302e+09,verified,Shawn,Suhrer,None,None,92354,Canine and Feline,False,ccd409a5-f53f-419a-bed3-de4101851e64,notified,2025-02-15 21:33:26.739308+00:00,EM_VETS_1,EM Vets Atlanta,14582726-100
109,f471f141-2775-4a3d-9724-363e2cc6bdaf,1.861020e+09,verified,Rasendu,Metzger,Duluth,GA,30068,Canine and Feline,False,09132144-cdcc-4f49-82ef-408eb65c5c6d,rejected,2025-02-14 18:46:23.864797+00:00,EM_VETS_4,EM Vets Dallas,14582726-103


In [73]:
# QA: Double check for: duplicate vet/event pairs & bad specialties 

# Count Duplicates for `vet_id` and `log_uuid`
duplicate_pairs = event_and_sub_df.duplicated(subset=["vet_id", "log_uuid"], keep=False).sum()
print(f"Duplicate vet_id & LOG_UUID pairs: {duplicate_pairs}")

# Count Specialty Occurrences and compare to https://www.avma.org/education/veterinary-specialties
# TODO: scrape this website to avoid manual checks in the future
specialty_counts = event_and_sub_df["specialty"].value_counts(dropna=False).reset_index()
specialty_counts.columns = ["Specialty", "Count"]
print("\nSpecialty Value Counts:")
display(specialty_counts)


Duplicate vet_id & LOG_UUID pairs: 0

Specialty Value Counts:


,Specialty,Count
0,Canine and Feline,67
1,Equine,13
2,None,7
3,Neurology,6
4,Beef cattle,4
5,Immunology,4
6,Other,3
7,Clinical pathology,2
8,Virology,2
9,Reptile and amphibian,1


*While we don't see any duplication of logs by vet, we do have "None" and "Other" Specialty categories that are not listed in the veterinarian specialties link that was provided. Because I am not sure whether unknown and "other" specialties should be excluded, I am including them as I assume the "Vet Targets" list came from a client who knows exactly who they want to target, regardless of the specialty we have on file for that vet. It would be good to confirm with stakeholders how they would like to handle this case going forward.*

# 3. Run SQL query to generate the emergency alert data format.

In [74]:
#final query, with necessary columns and filtering

final_query = """
    SELECT  
        V.vet_id, -- no VI.vet_ids match the targeting file
        COALESCE(V.first_name, EV.first_name) as first_name,
        COALESCE(V.last_name, EV.last_name) as last_name,
        EV.city,
        EV.state,
        COALESCE(V.zip_code, EV.zip) as zip,
        COALESCE(V.specialty, EV.specialty) as specialty, 
        VI.do_not_sell, 
        AL.event,
        AL.timestamp AS event_timestamp,
        ES.subscription_name, 
        ES.publisher, 
        ES.publisher_deal_id
    FROM VET_INFO VI 
    INNER JOIN VET V ON V.uuid = VI.uuid -- good join on foriegn keys
    LEFT JOIN EXTERNAL_VET_DATA EV ON EV.vet_id = V.vet_id -- assuming we can trust this join as most values align rather than VET_INFO
    INNER JOIN VET_EMERGENCY_ALERTS_LOG AL ON AL.vet_uuid = V.uuid
    INNER JOIN VET_EMERGENCY_SUBSCRIPTION ES on ES.uuid = AL.vet_emergency_subscription_uuid
    WHERE V.VET_ID IS NOT NULL -- final targeting file has to match VET_TARGETS which only has vet_id
        AND lower(trim(VI.vet_id_source)) = 'verified'
        AND do_not_sell != TRUE
    ORDER BY AL.timestamp
    ; 
    """

final_df = pd.read_sql(final_query, con=engine)
final_df

,vet_id,first_name,last_name,city,state,zip,specialty,do_not_sell,event,event_timestamp,subscription_name,publisher,publisher_deal_id
0,1649685096,Zhiyuan,Fogarty,None,None,10305,Canine and Feline,False,notified,2025-02-14 00:03:51.160376+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
1,1770013399,C,Pachinger,Augusta,GA,98908,Canine and Feline,False,notified,2025-02-14 00:17:59.865480+00:00,EM_VETS_1,EM Vets Atlanta,14582726-100
2,1508328085,Pearl,Zghouzi,San Francisco,CA,2911,Canine and Feline,False,notified,2025-02-14 01:10:37.622922+00:00,EM_VETS_1,EM Vets Atlanta,14582726-100
3,1972000673,Marilyn,Ceballos,Ocoee,FL,24018,Canine and Feline,False,notified,2025-02-14 03:05:36.716018+00:00,EM_VETS_12,EM Vets Chicago,14582726-111
4,1578068694,Renee,Ong,Tamarac,FL,15601,Canine and Feline,False,notified,2025-02-14 03:27:00.272145+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,1205153926,Tom,Javed,None,None,10701,Equine,False,notified,2025-02-16 17:05:01.909736+00:00,EM_VETS_4,EM Vets Dallas,14582726-103
89,1346737554,Levon,Aquino,Bronx,NY,30032,Canine and Feline,False,notified,2025-02-16 17:34:09.642603+00:00,EM_VETS_13,EM Vets Detroit,14582726-112
90,1477917313,Stanley,Miner,None,None,45215,Canine and Feline,False,rejected,2025-02-16 18:40:50.900371+00:00,EM_VETS_14,EM Vets Las Vegas,14582726-113
91,1477917313,Stanley,Miner,None,None,45215,Canine and Feline,False,notified,2025-02-16 18:40:50.919789+00:00,EM_VETS_13,EM Vets Detroit,14582726-112


# 4. Use Python to filter results based on the `Vet Targets` file and output the final processed data file.

In [78]:
# Generate final output file

# Load vet_targets to filter final_df
vet_targets_query = "SELECT vet_id FROM VET_TARGETS"
vet_targets_df = pd.read_sql(vet_targets_query, con=engine)

# Convert to a set for deduplication and fast filtering
vet_target_ids = set(vet_targets_df["vet_id"].dropna().astype(str)) 

# Filter final_df for vet_ids that exist in vet_targets
final_filtered_df = final_df[final_df["vet_id"].astype(str).isin(vet_target_ids)]

# Fix ZIP codes: Convert to string and pad with leading zeros
# NOTE: I noticed these zip codes don't align with the city/state. I am assuming the data is fake as opposed to problematic, 
#       and have not addressed zip issues in this notebook.
if "zip" in final_filtered_df.columns:
    final_filtered_df.loc[:, "zip"] = final_filtered_df["zip"].astype(str).str.zfill(5)

# Sort by event_timestamp
final_filtered_df = final_filtered_df.sort_values(by="event_timestamp")

# Save as a pipe (`|`) delimited file
output_file = "~/Downloads/vet_emergency_alerts_for_targets.txt"
final_filtered_df.to_csv(output_file, sep="|", index=False)

# Confirm file is saved
print(f"File saved successfully: {output_file}")
final_filtered_df


File saved successfully: ~/Downloads/vet_emergency_alerts_for_targets.txt


,vet_id,first_name,last_name,city,state,zip,specialty,do_not_sell,event,event_timestamp,subscription_name,publisher,publisher_deal_id
0,1649685096,Zhiyuan,Fogarty,None,None,10305,Canine and Feline,False,notified,2025-02-14 00:03:51.160376+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
1,1770013399,C,Pachinger,Augusta,GA,98908,Canine and Feline,False,notified,2025-02-14 00:17:59.865480+00:00,EM_VETS_1,EM Vets Atlanta,14582726-100
2,1508328085,Pearl,Zghouzi,San Francisco,CA,02911,Canine and Feline,False,notified,2025-02-14 01:10:37.622922+00:00,EM_VETS_1,EM Vets Atlanta,14582726-100
4,1578068694,Renee,Ong,Tamarac,FL,15601,Canine and Feline,False,notified,2025-02-14 03:27:00.272145+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
5,1174025710,Aravindhan,Naeem,Sioux Falls,SD,67601,Canine and Feline,False,notified,2025-02-14 03:37:11.175557+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
6,1174025710,Aravindhan,Naeem,Sioux Falls,SD,67601,Canine and Feline,False,accepted,2025-02-14 03:37:20.990314+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
7,1164650311,Brij,Adams,Clairton,PA,85118,Equine,False,notified,2025-02-14 04:37:34.157688+00:00,EM_VETS_3,EM Vets Los Angeles,14582726-102
8,1164650311,Brij,Adams,Clairton,PA,85118,Equine,False,rejected,2025-02-14 04:37:36.074911+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118
9,1326508777,Justin,Jama,Staten Island,NY,70817,Canine and Feline,False,notified,2025-02-14 08:20:34.248617+00:00,EM_VETS_12,EM Vets Chicago,14582726-111
10,1326508777,Justin,Jama,Staten Island,NY,70817,Canine and Feline,False,rejected,2025-02-14 08:20:36.684706+00:00,EM_VETS_19,EM Vets Charlotte,14582726-118


In [77]:
# Quick QA 

# Compute total records
total_records = len(final_filtered_df)

# Compute unique VET_ID count
unique_vet_ids = final_filtered_df["vet_id"].nunique()

# Compute count of missing values per column
missing_values = final_filtered_df.isna().sum()

# Create a summary DataFrame
summary_df = pd.DataFrame({
    "Metric": ["Total Records", "Unique VET_IDs"],
    "Count": [total_records, unique_vet_ids]
})

# Display results
print("Summary Statistics:")
print(summary_df)

print("\nMissing Values per Column:")
print(missing_values)


Summary Statistics:
           Metric  Count
0   Total Records     47
1  Unique VET_IDs     36

Missing Values per Column:
vet_id               0
vet_id_source        0
first_name           0
last_name            0
city                 4
state                4
zip                  0
specialty            0
do_not_sell          0
event                0
event_timestamp      0
subscription_name    0
publisher            0
publisher_deal_id    0
dtype: int64
